In [1]:
import pandas as pd

In [2]:
import urllib

In [3]:
import datetime

In [4]:
import time

In [7]:
time.strftime("%d%m%Y")

'10052016'

In [8]:
start = time.time()
print "hello Jalal Jahir what"
print "hello Jalal Jahir is a GREAT Guy"
end = time.time()
processtime = end - start
print "{0: .3}".format(processtime)
print float(processtime)

hello Jalal Jahir what
hello Jalal Jahir is a GREAT Guy
 0.0
0.0


In [9]:
import os

In [10]:
print os.getcwd()

J:\Staff\Jalal\Monthly_Emp_Update_Scratch\wa_esd_emp


In [11]:
current_dir = os.getcwd()

In [26]:
esd_sa_url = "https://fortress.wa.gov/esd/employmentdata/docs/economic-reports/wa-historical-employment-seasonally-adjusted.xlsx"
sa_filename = esd_sa_url[-49:-5]+"_"+time.strftime("%m%d%Y")+esd_sa_url[-5:]
def download_esd():
    esdfile = urllib.URLopener()
    esdfile.retrieve(esd_sa_url, os.getcwd()+"/"+ sa_filename)

In [31]:
download_esd()

In [29]:
help (urllib)

Help on module urllib:

NAME
    urllib - Open an arbitrary URL.

FILE
    c:\anaconda2\lib\urllib.py

DESCRIPTION
    See the following document for more info on URLs:
    "Names and Addresses, URIs, URLs, URNs, URCs", at
    http://www.w3.org/pub/WWW/Addressing/Overview.html
    
    See also the HTTP spec (from which the error codes are derived):
    "HTTP - Hypertext Transfer Protocol", at
    http://www.w3.org/pub/WWW/Protocols/
    
    Related standards and specs:
    - RFC1808: the "relative URL" spec. (authoritative status)
    - RFC1738 - the "URL standard". (authoritative status)
    - RFC1630 - the "URI spec". (informational status)
    
    The object returned by URLopener().open(file) will differ per
    protocol.  All you know is that is has methods read(), readline(),
    readlines(), fileno(), close() and info().  The read*(), fileno()
    and close() methods work like those of open files.
    The info() method returns a mimetools.Message object which can be
    used t

In [32]:
df = pd.read_excel(os.getcwd()+"/"+ sa_filename)

In [34]:
df.head()

,Washington State Quarterly Benchmarked,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 307,Unnamed: 308,Unnamed: 309,Unnamed: 310,Unnamed: 311,Unnamed: 312,Unnamed: 313,Unnamed: 314,Unnamed: 315,Unnamed: 316
0,NAICS CELL,NAICS INDUSTRY,1990-01-01 00:00:00,1990-02-01 00:00:00,1990-03-01 00:00:00,1990-04-01 00:00:00,1990-05-01 00:00:00,1990-06-01 00:00:00,1990-07-01 00:00:00,1990-08-01 00:00:00,...,2015-06-01 00:00:00,2015-07-01 00:00:00,2015-08-01 00:00:00,2015-09-01 00:00:00,2015-10-01 00:00:00,2015-11-01 00:00:00,2015-12-01 00:00:00,2016-01-01 00:00:00,2016-02-01 00:00:00,2016-03-01 00:00:00
1,00000000,Total Nonfarm,2117500,2121000,2129700,2137000,2145100,2154900,2154100,2165200,...,3155900,3157700,3164500,3170600,3175900,3190600,3197800,3211800,3225200,3234200
2,05000000,Total Private,1727500,1726600,1737200,1743000,1747900,1757700,1758200,1762800,...,2594900,2594200,2597000,2602100,2611100,2623200,2629100,2642800,2655300,2663700
3,06000000,Goods Producing,461400,459400,464900,465900,467000,469100,469700,470100,...,472100,472100,469600,468600,467500,470600,472300,473300,475700,477400
4,10000000,Mining and Logging,13000,12800,12800,12800,12800,12700,12100,12600,...,6300,6200,6100,6000,5800,5800,5800,5800,5700,5800
